In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import csv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train_keys.pkl",compression="zip")

In [ ]:
train.head()

,index,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID,keywords
0,0,pete the cat bedtime blues doll 14 5 inch,pete the cat is the coolest most popular cat i...,pete the cat bedtime blues plush doll based on...,merrymakers,0,cuddle measures | pj s | james dean | popular ...
1,1,the new yorker nyhm014 refrigerator magnet 2 x...,the new yorker handsome cello wrapped hard mag...,cat in a tea cup by new yorker cover artist gu...,the new yorker,1,new yorker nyhm014 refrigerator magnet | artis...
2,2,the ultimate self sufficiency handbook a compl...,NaN,skyhorse publishing,imusti,2,harvest raising animals | ultimate self suffic...
3,3,amway nutrilite kids chewable iron tablets 100,NaN,nutrilite kids chewable iron tablets quantity ...,amway,3,amway nutrilite kids | tablets quantity | nutr...
4,4,teacher planner company a4 6 lesson academic t...,NaN,NaN,NaN,4,teacher planner company a4 | academic teacher ...


In [ ]:
train.BRAND.fillna("OTHER_BRANDS",inplace=True)

In [ ]:
lol = train[['BRAND','BROWSE_NODE_ID']].groupby("BRAND")
lol.head()

,BRAND,BROWSE_NODE_ID
0,merrymakers,0
1,the new yorker,1
2,imusti,2
3,amway,3
4,OTHER_BRANDS,4
...,...,...
2902995,ozpolish,30348
2902996,rhv,45936
2903000,pawan textiles,465
2903012,torr to,1044933


In [ ]:
lol1 = lol.nunique()
lol1.head(10)

,BROWSE_NODE_ID
BRAND,
,57
0,3
0 degree,15
000 fashions,1
001 girls shopping,1
007 bond hut,1
007 fragrances,1
00ra with as logo,4
02 cool mist n sip,1


In [ ]:
lol1.reset_index(inplace=True)
lol1.head()

,BRAND,BROWSE_NODE_ID
0,,57
1,0,3
2,0 degree,15
3,000 fashions,1
4,001 girls shopping,1


In [ ]:
lol1 = lol1[["BRAND","BROWSE_NODE_ID"]]
lol1.rename(columns = {"BROWSE_NODE_ID":"nunique_nodes"},inplace=True)
lol1.head()

,BRAND,nunique_nodes
0,,57
1,0,3
2,0 degree,15
3,000 fashions,1
4,001 girls shopping,1


In [ ]:
data_lol = pd.merge(train,lol1, left_on='BRAND', right_on='BRAND',how='left')

In [ ]:
data_lol.head()

,index,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID,keywords,nunique_nodes
0,0,pete the cat bedtime blues doll 14 5 inch,pete the cat is the coolest most popular cat i...,pete the cat bedtime blues plush doll based on...,merrymakers,0,cuddle measures | pj s | james dean | popular ...,2
1,1,the new yorker nyhm014 refrigerator magnet 2 x...,the new yorker handsome cello wrapped hard mag...,cat in a tea cup by new yorker cover artist gu...,the new yorker,1,new yorker nyhm014 refrigerator magnet | artis...,3
2,2,the ultimate self sufficiency handbook a compl...,NaN,skyhorse publishing,imusti,2,harvest raising animals | ultimate self suffic...,25
3,3,amway nutrilite kids chewable iron tablets 100,NaN,nutrilite kids chewable iron tablets quantity ...,amway,3,amway nutrilite kids | tablets quantity | nutr...,31
4,4,teacher planner company a4 6 lesson academic t...,NaN,NaN,OTHER_BRANDS,4,teacher planner company a4 | academic teacher ...,3846


In [ ]:
data_lol.BROWSE_NODE_ID.value_counts()

1045       215698
5           70318
1251        51929
1052        45553
4           34169
            ...  
279112          1
822060          1
1337233         1
2136811         1
248936          1
Name: BROWSE_NODE_ID, Length: 9919, dtype: int64

In [ ]:
new_data = data_lol[data_lol.nunique_nodes == 1][["BRAND","BROWSE_NODE_ID"]]
new_data = new_data.drop_duplicates(subset=["BRAND"])
new_data.head()

,BRAND,BROWSE_NODE_ID
7,hungry brain,7
39,panter company,39
76,scent shield,76
79,itw consumer devcon,79
83,bazzill,83


In [ ]:
new_data.reset_index(drop=True,inplace=True)
new_data.to_csv("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/mappings.csv",index=False)

In [ ]:
len(new_data)

140945

In [ ]:
values = data_lol[data_lol.nunique_nodes == 1].BRAND.values

with open("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/lone_brands.txt", 'w') as file:
        for row in values:
            file.write(row+'\n')

In [ ]:
data_lol = data_lol[data_lol.nunique_nodes>1]
len(data_lol)

2631811

In [ ]:
len(df)

In [ ]:
del train
del lol
del lol1

In [ ]:
data_lol.to_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train_limited_brands.pkl",compression="zip")

In [ ]:
test = pd.read_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/test.pkl",compression="zip")

In [ ]:
print(test['BRAND'])

In [ ]:
values = list(set(list(values)))

In [ ]:
train[train.BRAND == "hungry brain"]['BROWSE_NODE_ID'].values[0]

7

In [ ]:
dic={}
for i in tqdm(values):
    node= train[train.BRAND == i]['BROWSE_NODE_ID'].values[0]
    dic[i]=node
dic

In [ ]:
import json
import numpy as np

def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

with open('/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/json_brand.txt', 'w') as fp:
    json.dump(dic, fp, default=np_encoder)

In [ ]:
for i in values:
  print(i, ":", train.BRAND[train.BRAND==i].count())

In [ ]:
test.BRAND.fillna("OTHER_BRANDS",inplace=True)

In [ ]:
values.map(test['BRAND'])

In [3]:
data = pd.read_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train_limited_brands.pkl",compression="zip")
data.reset_index(drop=True,inplace=True)

In [4]:
sum(data.BROWSE_NODE_ID.value_counts().values>1000)

462

In [5]:
nodes = list(data.BROWSE_NODE_ID.value_counts().index[:462])

In [6]:
indexes = []
main_df = pd.DataFrame()
for i in tqdm(nodes):
    df = data[data.BROWSE_NODE_ID == i]
    indexes.extend(list(df.index))
    main_df = main_df.append(df.sample(min(len(df),1000)))

In [7]:
data.drop(indexes,inplace=True)
main_df = main_df.append(data)
main_df.reset_index(drop=True,inplace=True)

In [8]:
main_df.to_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/new_data.pkl",compression="zip")

In [9]:
len(main_df)

1287826

In [ ]:
nodes = list(data.BROWSE_NODE_ID.value_counts().index)[450:]

In [ ]:
len(nodes)

7419